Sources

#### Projet End-to-End (Pas NHL API)
https://medium.com/coinmonks/4-718-using-machine-learning-to-bet-on-the-nhl-25d16649cd52

#### Projet Live Feed (NHL API)
##### Utilise le live feed pour shot sur gardien
https://towardsdatascience.com/nhl-analytics-with-python-6390c5d3206d

#### Docu NHL API
https://github.com/dword4/nhlapi

Ancienne Doc:
https://webcache.googleusercontent.com/search?q=cache:ILYmzDuxW70J:https://github.com/dword4/nhlapi+&cd=1&hl=en&ct=clnk&gl=ca

In [12]:
# Check Python Version
from platform import python_version
import pandas as pd
import requests  
from pandas.io.json import json_normalize
import datetime

pd.set_option('display.max_columns', None)

#print(pd.show_versions())

In [13]:
def extract_standings(gameID):
    
    '''
    This function takes the gameID as an input and returns a dataframe that contains
    the teamSkaterStats info for the home/away teams for that specific game.
    '''
    
    r = requests.get(url='https://statsapi.web.nhl.com/api/v1/game/{}/boxscore'.format(gameID))
    data = r.json()
    df = pd.json_normalize(data = data['teams'])
    df = df.filter(regex='teamSkaterStats')
    df['gameID'] = gameID
    
    return df

### import game data

In [35]:
start_date = datetime.date(2019, 4, 21)
end_date = datetime.date(2019, 4, 23)
delta = datetime.timedelta(days=1)

data_list = []

while start_date <= end_date:
    
    r = requests.get(url='https://statsapi.web.nhl.com/api/v1/schedule?date=' + start_date.strftime("%Y-%m-%d"))
    data = r.json()
    
    df = pd.json_normalize(data = data['dates'], record_path='games', meta=['date'])
    
    # On ajoute des statistiques au niveau des matchs
    # Le if sert a skipper les journees sans matchs
    if df.empty==False :
        pd_list = []
        for games in df['gamePk']:
            tmp = extract_standings(gameID=games)
            pd_list.append(tmp)

        pd_tmp = pd.concat(pd_list, sort='False', ignore_index=True)
        df.merge(pd_tmp, left_on='gamePk', right_on='gameID', how='left')

        data_list.append(df)
        
        start_date += delta
    else:
        start_date += delta

In [32]:
df = pd.concat(data_list, sort='False', ignore_index=True)
df.to_csv('../../data/games.csv')

In [33]:
df.head(5)

,content.link,date,gameDate,gamePk,gameType,link,season,status.abstractGameState,status.codedGameState,status.detailedState,status.startTimeTBD,status.statusCode,teams.away.leagueRecord.losses,teams.away.leagueRecord.ot,teams.away.leagueRecord.type,teams.away.leagueRecord.wins,teams.away.score,teams.away.team.id,teams.away.team.link,teams.away.team.name,teams.home.leagueRecord.losses,teams.home.leagueRecord.ot,teams.home.leagueRecord.type,teams.home.leagueRecord.wins,teams.home.score,teams.home.team.id,teams.home.team.link,teams.home.team.name,venue.id,venue.link,venue.name
0,/api/v1/game/2017030316/content,2018-05-21,2018-05-22T00:00:00Z,2017030316,P,/api/v1/game/2017030316/feed/live,20172018,Final,7,Final,False,7,5,NaN,league,11,0,14,/api/v1/teams/14,Tampa Bay Lightning,7,NaN,league,11,3,15,/api/v1/teams/15,Washington Capitals,NaN,/api/v1/venues/null,Verizon Center
1,/api/v1/game/2017030317/content,2018-05-23,2018-05-24T00:00:00Z,2017030317,P,/api/v1/game/2017030317/feed/live,20172018,Final,7,Final,False,7,7,NaN,league,12,4,15,/api/v1/teams/15,Washington Capitals,6,NaN,league,11,0,14,/api/v1/teams/14,Tampa Bay Lightning,5017.0,/api/v1/venues/5017,Amalie Arena
2,/api/v1/game/2017030411/content,2018-05-28,2018-05-29T00:00:00Z,2017030411,P,/api/v1/game/2017030411/feed/live,20172018,Final,7,Final,False,7,8,NaN,league,12,4,15,/api/v1/teams/15,Washington Capitals,3,NaN,league,13,6,54,/api/v1/teams/54,Vegas Golden Knights,5178.0,/api/v1/venues/5178,T-Mobile Arena
3,/api/v1/game/2017030412/content,2018-05-30,2018-05-31T00:00:00Z,2017030412,P,/api/v1/game/2017030412/feed/live,20172018,Final,7,Final,False,7,8,NaN,league,13,3,15,/api/v1/teams/15,Washington Capitals,4,NaN,league,13,2,54,/api/v1/teams/54,Vegas Golden Knights,5178.0,/api/v1/venues/5178,T-Mobile Arena
4,/api/v1/game/2017030413/content,2018-06-02,2018-06-03T00:00:00Z,2017030413,P,/api/v1/game/2017030413/feed/live,20172018,Final,7,Final,False,7,5,NaN,league,13,1,54,/api/v1/teams/54,Vegas Golden Knights,8,NaN,league,14,3,15,/api/v1/teams/15,Washington Capitals,NaN,/api/v1/venues/null,Verizon Center


### Import teams data

In [12]:
url = 'https://statsapi.web.nhl.com/api/v1/teams'
r = requests.get(url)
json_data = r.json()
teams = json_normalize(json_data['teams'], sep = "_")

teams.to_csv('../../data/teams.csv')

C:\Users\athompson-leduc\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  after removing the cwd from sys.path.
